# Exploring pestpp-ies and pestpp-mou results with `pyemu.Pst`

To help with processing the insane amount of output generated by pestpp-ies and pestpp-mou, `pyemu.Pst` includes a results-handling helper class to hide all of the nastiness associated with finding specific iteration/generation results.  Let see how this works... 

In [ ]:
import os
import pyemu

The result handlers are based on having a stand-alone directory that holds the results of a completed pestpp-ies/pestpp-mou run. We have added a few of these directories to the pyemu repo for testing and for this example - your welcome!

In [ ]:
m_d_ies1 = os.path.join("..","autotest","pst","master_ies1")
m_d_ies2 = os.path.join("..","autotest","pst","master_ies2")
m_d_mou1 = os.path.join("..","autotest","pst","zdt1_bin")
m_d_mou2 = os.path.join("..","autotest","pst","zdt1_ascii")
assert os.path.exists(m_d_ies1)
assert os.path.exists(m_d_ies2)
assert os.path.exists(m_d_mou1)
assert os.path.exists(m_d_mou2)
print([f for f in os.listdir(m_d_ies1) if f.endswith(".pst")])
print([f for f in os.listdir(m_d_ies2) if f.endswith(".pst")])

## pestpp-ies

We see the `m_d_ies11` contains a single control file, while `m_d_ies2` has many...

Like usual, we need to load the control file into a `Pst` instance.  We can assume that even tho `m_d_ies2` has lots of control files, they have the same numbers of pars and obs (altho this is not required by the result handler), so maybe this collection of control files represents different experiments that have been run.  Let's just use the single control in `m_d_ies1`:

In [ ]:
pst = pyemu.Pst(os.path.join(m_d_ies1,"pest.pst"))

In [ ]:
pst

To use the result-handler, we need to register one or more results directories with `Pst` via the `add_results()` method, or by passing the result dir path to the constructor (like `pyemu.Pst(os.path.join(m_d_ies1,"pest.pst"),result_dir=m_d_ies1)`.  But by default, the directory containing the control file is used, so this case we are good to go...

Now for the fun part! We can access the pestpp-ies results in that directory via the `.ies` attribute:

In [ ]:
pst.ies

Lets say we want to see the prior parameter ensemble.  We do this via the `paren` (for parameter ensemble) with a '0' suffix for the 0th iteration (ie the prior):

In [ ]:
pst.ies.paren0.head()

The result handler uses lazy evaluation and holds previously access files in memory:

In [ ]:
pst.ies.files_loaded

This lazy evaluation means accessing the prior par ensemble next time is super cheap b/c its being held in memory..
How about the correspoding obs ensemble:



In [ ]:
pst.ies.obsen0.head()

In [ ]:
pst.ies.files_loaded

What if we want to see all of the par and obs ensembles across all iterations?  All we need to do now is leave off an iteration suffix from the `paren` attribute, and we will get back a multi-index dataframe:

In [ ]:
pst.ies.paren

How about noise and weight ensembles you say?  easy as...

In [ ]:
pst.ies.noise.head()

In [ ]:
pst.ies.weights.head()

yeah, thats pretty awesome....

we can also access the parameter-change summaries:

In [ ]:
pst.ies.pcs

And the prior-data-conflict information:

In [ ]:
pst.ies.pdc

There's not any prior-data conflict in these examples, but we can check that all the `pdc` files have been loaded:

In [ ]:
[f for f in pst.ies.files_loaded if "pdc" in f]

And the phi summary information:

In [ ]:
pst.ies.phiactual.head()

In [ ]:
pst.ies.phimeas.head()

And we can also load the rmr file into a dataframe if you are interested in such things:

In [ ]:
pst.ies.rmr

## pestpp-mou
Things work in a similar way for pestpp-mou:

In [ ]:
print([f for f in os.listdir(m_d_mou1) if f.endswith(".pst")])
print([f for f in os.listdir(m_d_mou2) if f.endswith(".pst")])

In [ ]:
pst = pyemu.Pst(os.path.join(m_d_mou1,"zdt1.pst"))

Like with the pestpp-ies handler, we can access the individual populations or the combined populations thru the `obspop` and `dvpop` attributes for the observation population and decision-variable population, respectively:

In [ ]:
pst.mou.obspop0

In [ ]:
pst.mou.dvpop

The archive versions of those are available as:

In [ ]:
pst.mou.archivedvpop0

In [ ]:
pst.mou.archivedvpop

And the chance populations (if applicable) are also available:

In [ ]:
pst.mou.chanceobspop

The pareto summary and pareto archive summaries are accessible like this:

In [ ]:
pst.mou.paretosum

In [ ]:
pst.mou.paretosum_archive

In [ ]:
pst.mou.files_loaded

## Dealing with multiple results diretories

This is getting deep!  While not a common use-case yet, we are working toward being able to automatically/programmatically compared two or more pestpp-ies/pestpp-mou analyses, where each analysis is stored in a seperate directory...

In [ ]:
print([f for f in os.listdir(m_d_ies1) if f.endswith(".pst")])
print([f for f in os.listdir(m_d_ies2) if f.endswith(".pst")])

In [ ]:
pst = pyemu.Pst(os.path.join(m_d_ies1,"pest.pst"))

When using multiple directories, you can pass an optional "cases" argument in the case where you have named the control file a different name in different results directories.  In this example, the first directory uses the case "pest" and we will use the case "test" in the second directory (note the "test" case run used binary output files - the results handler doesnt care!):

In [ ]:
pst.add_results([m_d_ies2],cases=["test"])

The results directories are now accessible via the `r` prefix attribute, where `r0` the first dir, `r1 is the second, etc

In [ ]:
pst.r0.ies.paren

You can also access the results via the directory name:

In [ ]:
pst.master_ies2.ies.obsen

In [ ]:
pst.master_ies2.ies.files_loaded

Or you can get a list of the ies or mou specific handlers for all the directories (in the ordered they were passed to `Pst.add_results()`):

In [ ]:
ies_all = pst.ies

In [ ]:
ies_all

In [ ]:
ies_all[0].phiactual

Enjoy!